In [1]:
import pandas as pd
from glob import glob
import numpy as np
import copy
path = 'C:/Users/0630r/inha2/Research/Dart/Dart_file/real_code/pqt_file/'

In [2]:
df_bid = pd.read_parquet(path + 'orign_bid.pqt', engine = 'pyarrow') # 전 bid 원본 데이터 사용
df_bid

,등록유형,공고게시일자,입찰공고번호,공고기관명,입찰업체사업자등록번호,수요기관구분,참가수,입찰금액,낙찰여부,업종제한여부,업종제한내용,지역제한내용
0,나라장터(G2B),20200102,20191241947,조달청 인천지방조달청,1218211789,공기업,1.0,2.380000e+06,Y,N,None,None
1,나라장터(G2B),20200102,20191241963,조달청 인천지방조달청,1048620001,공기업,1.0,2.380200e+06,Y,N,None,None
2,나라장터(G2B),20200102,20200100170,한국보훈복지의료공단 중앙보훈병원,2148753956,준정부기관,1.0,6.834780e+08,Y,N,None,None
3,나라장터(G2B),20200102,20200100178,한국보훈복지의료공단 중앙보훈병원,1138657370,준정부기관,1.0,1.991412e+08,Y,N,None,None
4,나라장터(G2B),20200102,20191236817,조달청 대구지방조달청,2128208426,지자체,1.0,1.210000e+08,Y,N,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
14170523,자체전자조달시스템,20221226,202296021,한국도로공사,1278609047,None,NaN,2.499860e+09,N,None,None,None
14170524,자체전자조달시스템,20221226,202296021,한국도로공사,5020144383,None,NaN,2.506757e+09,N,None,None,None
14170525,자체전자조달시스템,20221226,202296021,한국도로공사,4108620323,None,NaN,2.520860e+09,N,None,None,None
14170526,자체전자조달시스템,20221226,202296021,한국도로공사,5888801225,None,NaN,2.527308e+09,N,None,None,None


In [3]:
df_bid['공고게시일자'] = (df_bid['공고게시일자'].values / 1e2).astype(int) # year 컬럼 생성
df_bid['연도'] = (df_bid['공고게시일자'].values / 1e2).astype(int) # year 컬럼 생성
df_bid['분기'] = df_bid['공고게시일자'].apply(lambda x: int(str(x)[4:]))
df_bid['분기'] = (df_bid['분기'] - 1) // 3 + 1 
df_bid['분기'] = df_bid['분기'].apply(lambda x: str(x)+'Q')
df_bid.drop(columns = '공고게시일자', inplace = True)
df_bid = df_bid.reindex(columns = ['연도', '분기', '입찰업체사업자등록번호', '수요기관구분', '참가수', '입찰금액', '낙찰여부'])
df_bid.dropna(inplace = True)
df_bid

,연도,분기,입찰업체사업자등록번호,수요기관구분,참가수,입찰금액,낙찰여부
0,2020,1Q,1218211789,공기업,1.0,2380000.0,Y
1,2020,1Q,1048620001,공기업,1.0,2380200.0,Y
2,2020,1Q,2148753956,준정부기관,1.0,683477980.0,Y
3,2020,1Q,1138657370,준정부기관,1.0,199141220.0,Y
4,2020,1Q,2128208426,지자체,1.0,121000000.0,Y
...,...,...,...,...,...,...,...
14168278,2022,4Q,6208151748,교육기관,3.0,1400.0,Y
14168279,2022,4Q,3630501453,교육기관,3.0,27200.0,N
14168280,2022,4Q,6100109969,교육기관,3.0,31000.0,N
14168281,2022,4Q,5071283543,교육기관,2.0,1940.0,Y


In [4]:
df_bid.rename(columns = {'입찰업체사업자등록번호':'사업자등록번호'}, inplace = True)
df_bid

,연도,분기,사업자등록번호,수요기관구분,참가수,입찰금액,낙찰여부
0,2020,1Q,1218211789,공기업,1.0,2380000.0,Y
1,2020,1Q,1048620001,공기업,1.0,2380200.0,Y
2,2020,1Q,2148753956,준정부기관,1.0,683477980.0,Y
3,2020,1Q,1138657370,준정부기관,1.0,199141220.0,Y
4,2020,1Q,2128208426,지자체,1.0,121000000.0,Y
...,...,...,...,...,...,...,...
14168278,2022,4Q,6208151748,교육기관,3.0,1400.0,Y
14168279,2022,4Q,3630501453,교육기관,3.0,27200.0,N
14168280,2022,4Q,6100109969,교육기관,3.0,31000.0,N
14168281,2022,4Q,5071283543,교육기관,2.0,1940.0,Y


In [5]:
## 기업, 연도, 분기, 기관별 1.낙찰수, 2. 입찰금액sum, 3. 입찰 참여 수 4. 낙찰사업 참여자수 

In [6]:
## 입찰 횟수
df_size = df_bid.groupby(['연도','분기', '사업자등록번호']).size()
df_size.name = '입찰횟수'
df_size = pd.DataFrame(df_size)
df_size = df_size.reset_index()
df_size

,연도,분기,사업자등록번호,입찰횟수
0,2020,1Q,1010116082,12
1,2020,1Q,1010116587,1
2,2020,1Q,1010118644,1
3,2020,1Q,1010145888,7
4,2020,1Q,1010146043,109
...,...,...,...,...
462721,2022,4Q,8998701899,29
462722,2022,4Q,8998800373,2
462723,2022,4Q,8998800524,7
462724,2022,4Q,8998801692,11


In [7]:
## 낙찰횟수, 낙찰사업 평균참여수, 낙찰금액
df_Y = df_bid.loc[df_bid.낙찰여부 == 'Y', :]
df_Y = df_Y.groupby(['연도', '분기', '사업자등록번호']).agg({'참가수': np.mean, '입찰금액': np.sum, '낙찰여부': np.size})
df_Y.reset_index(inplace = True)
df_Y.rename(columns = {'참가수': '낙찰사업평균참여수', '낙찰여부': '낙찰수', '입찰금액': '낙찰금액'}, inplace = True)
df_Y

,연도,분기,사업자등록번호,낙찰사업평균참여수,낙찰금액,낙찰수
0,2020,1Q,1010145888,4.50,5.407900e+05,2
1,2020,1Q,1010148814,2.00,2.860000e+07,1
2,2020,1Q,1010154956,296.00,2.119200e+02,1
3,2020,1Q,1010211051,3.00,2.000100e+08,1
4,2020,1Q,1010266027,9.00,1.730100e+08,1
...,...,...,...,...,...,...
122471,2022,4Q,8989101379,176.00,1.085036e+07,1
122472,2022,4Q,8998100995,3.00,2.928600e+07,1
122473,2022,4Q,8998101000,28.00,1.256629e+08,1
122474,2022,4Q,8998601927,29.25,2.603552e+08,4


In [8]:
col = ['연도', '분기', '사업자등록번호']
bid_tt = pd.merge(df_size, df_Y, how = 'left', on = col)
bid_tt.fillna(0, inplace = True)
bid_tt

,연도,분기,사업자등록번호,입찰횟수,낙찰사업평균참여수,낙찰금액,낙찰수
0,2020,1Q,1010116082,12,0.0,0.0,0.0
1,2020,1Q,1010116587,1,0.0,0.0,0.0
2,2020,1Q,1010118644,1,0.0,0.0,0.0
3,2020,1Q,1010145888,7,4.5,540790.0,2.0
4,2020,1Q,1010146043,109,0.0,0.0,0.0
...,...,...,...,...,...,...,...
462721,2022,4Q,8998701899,29,0.0,0.0,0.0
462722,2022,4Q,8998800373,2,0.0,0.0,0.0
462723,2022,4Q,8998800524,7,32.0,106181490.0,1.0
462724,2022,4Q,8998801692,11,0.0,0.0,0.0


In [9]:
bid_tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462726 entries, 0 to 462725
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   연도         462726 non-null  int64  
 1   분기         462726 non-null  object 
 2   사업자등록번호    462726 non-null  int64  
 3   입찰횟수       462726 non-null  int64  
 4   낙찰사업평균참여수  462726 non-null  float64
 5   낙찰금액       462726 non-null  float64
 6   낙찰수        462726 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 28.2+ MB


In [10]:
bid_tt.to_parquet('bid_nodemand.pqt') # 수요기관 없는 데이터 저장
# 완    

In [ ]:
# bid_nodemand 한 개 저장